# Исследование надежности заемщиков

### Цель исследования:
- Определить факторы, влияющие на факт погашения кредита в срок


### Задачи исследования:

- Предобработать данные для проведения дальнейшего анализа (удалить дубликаты, пустые строчки, произвести лемматизацию для определения наиболее часто выбранной цели кредита, категоризировать клиентов)
- Выделить факторы, которые могут влиять на факт погашения кредита в срок
- Составить рекомендации для кредитного комитета в банке

### Шаг 1. Изучение общей информации о данных 

In [1]:
! pip install pymystem3

In [2]:
import pandas as pd
# библиотеки ниже нужны, так как будем заниматься лемматизацией
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

Смотрим общую информацию о таблице. Видим, что пропущены значения в столбцах days_employed,  total_income

In [3]:
data = pd.read_csv('C:/Users/darya/Documents/Datasets/df_debts.csv')
data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Смотрим на сами данные. Тут видим, что в каждом столбце есть неправильные значения. Например, в столбце education необходимо привести все к одному регистру

In [4]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Далее мы посмотрели уникальные значения по каждому столбцу для того, чтобы наметить себе план работ. 

Какие изменения необходимо провести:
- children - удалить или модицифировать артефакты "-1 детей" и "20 детей"
- days_employed - заполнить пропуски (так как для дальнейшего анализа этот столбец не нужен, с отрицательными значениями ничего делать не будем)
- dob_years - решить, что сделать со значением "0"
- education - избавиться от разного регистра
- family_status_id - проверить, соответствуют ли значения в этом поле значениям в family_status
- total_income - заполнить пропуски
- purpose - лемматизировать
- gender - удалить или модицифировать артефакт "XNA"


### Шаг 2. Предобработка данных


### Обработка пропусков


Мы обнаружили два поля, в которых есть пустые значения - days_employed и total_income. 
Так как далее поле days_employed не используется, просто заполняем его нулями. 
В то же время, пропуски в total_income просто так нулями заполнять нельзя. Поэтому в случае с total_income сначала проверим, у какой группы появились наллы в этом поле 

In [5]:
#заполнили нулями, тк этот столбец в дальнейшем анализе не нужен
data['days_employed'].fillna('0', inplace=True) 

In [6]:
#превратим days_employed в число
data['days_employed'] = data['days_employed'].astype('int') 

Теперь приступим к заполнению наллов в поле total_income
Дальше решили сгруппировать по разным колонкам, чтобы попробовать найти причину, почему total_income может быть заполнен наллом

In [7]:
#здесь сначала проверили, как распределяются наны в total_income по income_type
data[data['total_income'].isnull()].groupby('income_type').count() 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,debt,total_income,purpose
income_type,,,,,,,,,,,
госслужащий,147,147,147,147,147,147,147,147,147,0,147
компаньон,508,508,508,508,508,508,508,508,508,0,508
пенсионер,413,413,413,413,413,413,413,413,413,0,413
предприниматель,1,1,1,1,1,1,1,1,1,0,1
сотрудник,1105,1105,1105,1105,1105,1105,1105,1105,1105,0,1105


Сгруппируем по столбцу days_employed

In [8]:
data[data['total_income'].isnull()].groupby('days_employed').count()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
days_employed,,,,,,,,,,,
0,2174,2174,2174,2174,2174,2174,2174,2174,2174,0,2174


Так мы выяснили, что days_imployed является наллом у людей, у которых days_employed = 0, то есть, по всей видимости, они никогда не работали

Сначала была идея заполнить все наллы медианным значением по income_type, но сейчас мы видим, что лучше их превратить в нули, как в случае с первыми пропусками, так как иначе мы им слишком завысим показатели

In [9]:
data['total_income'].fillna('0', inplace=True)

### Вывод

Таким образом, в данном шаге мы избавились от null-значений в таблице. 

Касаемо причин пропуска можем выделить следующие: 
- незаполнение анкеты до конца
- некоторые вопросы были необязательными
- если сбор данных проводился через сайт или приложение, то система могла сама подставлять ответы. 

Рекомендовали бы помечать те ответы, которые являются обязательными, звездочкой или, если анкета находится на сайте или в приложении, не пускать на дальнейший этап без корректно заполненных даннных.


### Замена типа данных

В наборе данных представлены такие типы данных, как строки, int, float. Давайте посмотрим, что необходимо заменить.
Очевидно, что если значение может быть только целым (например, количество людей), то эти данные необходимо превратить в int
Посмотрим еще раз, какие типы данных у нас есть. Итого из того, что надо заменить, остается только total_income - превращаем его в int


In [10]:
data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int32 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  object
 11  purpose           21525 non-null  object
dtypes: int32(1), int64(5), object(6)
memory usage: 1.9+ MB


In [11]:
data['total_income'] = data['total_income'].astype('int')

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int32 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int32 
 11  purpose           21525 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 1.8+ MB


### Обработка дубликатов

Первое, что необходимо обработать, - это возможное появление букв разного регистра в таблице. Обработаем все строковые поля сразу циклом

In [13]:
for column in ['education', 'family_status', 'gender', 'income_type', 'purpose']:
    data[column] = data[column].str.lower()

Посмотрим на уникальные значения в поле gender

In [14]:
data['gender'].unique() #видим xna

array(['f', 'm', 'xna'], dtype=object)

In [15]:
data.groupby('gender').count() 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,income_type,debt,total_income,purpose
gender,,,,,,,,,,,
f,14236,14236,14236,14236,14236,14236,14236,14236,14236,14236,14236
m,7288,7288,7288,7288,7288,7288,7288,7288,7288,7288,7288
xna,1,1,1,1,1,1,1,1,1,1,1


xna выбрал всего 1 человек, это меньше стат.погрешности, поэтому удаляем

In [16]:
data = data.loc[data['gender'] != 'xna']

Теперь посмотрим на другие столбцы с некорректными значениями. В dob_years мы видим 101 человека, чей возраст 0 лет. Вряд ли грудничкам понадобился кредит, поэтому эти значения необходимо заменить или удалить

Далее выясняем, какой из вариантов подойдет больше всего

In [17]:
data.groupby('dob_years').count() 

,children,days_employed,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
dob_years,,,,,,,,,,,
0,101,101,101,101,101,101,101,101,101,101,101
19,14,14,14,14,14,14,14,14,14,14,14
20,51,51,51,51,51,51,51,51,51,51,51
21,111,111,111,111,111,111,111,111,111,111,111
22,183,183,183,183,183,183,183,183,183,183,183
23,254,254,254,254,254,254,254,254,254,254,254
24,263,263,263,263,263,263,263,263,263,263,263
25,357,357,357,357,357,357,357,357,357,357,357
26,408,408,408,408,408,408,408,408,408,408,408


Далее посмотрели группировку людей, у которых dob_years = 0 по другим параметрам. Выявили, что нет никакого маркера, чтобы отнести их к какой-то группе: они равномерно распределены по income_type, purpose, education, gender, определенной связи с другими столбцами мы не нашли, пришли к выводу, что это или какая-то системная ошибка, или данные собрались некорректно. Поэтому эти строки решили удалить

In [18]:
data[data['dob_years'] == 0].groupby('income_type').count() #так мы группировали по всем столбцам

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,debt,total_income,purpose
income_type,,,,,,,,,,,
госслужащий,6,6,6,6,6,6,6,6,6,6,6
компаньон,20,20,20,20,20,20,20,20,20,20,20
пенсионер,20,20,20,20,20,20,20,20,20,20,20
сотрудник,55,55,55,55,55,55,55,55,55,55,55


Удаляем строки с 0

In [19]:
data = data.loc[data['dob_years'] != 0]

In [20]:
data.info() #проверяем, что строки удалились

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21423 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21423 non-null  int64 
 1   days_employed     21423 non-null  int32 
 2   dob_years         21423 non-null  int64 
 3   education         21423 non-null  object
 4   education_id      21423 non-null  int64 
 5   family_status     21423 non-null  object
 6   family_status_id  21423 non-null  int64 
 7   gender            21423 non-null  object
 8   income_type       21423 non-null  object
 9   debt              21423 non-null  int64 
 10  total_income      21423 non-null  int32 
 11  purpose           21423 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 2.0+ MB


Теперь необходимо удалить дубликаты

In [21]:
data.duplicated().sum()

71

Дубликатов 71. Удаляем их

In [22]:
data = data.drop_duplicates()

In [23]:
data.info() #проверяем, что дубликатов нет

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21352 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21352 non-null  int64 
 1   days_employed     21352 non-null  int32 
 2   dob_years         21352 non-null  int64 
 3   education         21352 non-null  object
 4   education_id      21352 non-null  int64 
 5   family_status     21352 non-null  object
 6   family_status_id  21352 non-null  int64 
 7   gender            21352 non-null  object
 8   income_type       21352 non-null  object
 9   debt              21352 non-null  int64 
 10  total_income      21352 non-null  int32 
 11  purpose           21352 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Таким образом, мы обработали дубликаты, которые попадались в таблице. Привели данные к одному регистру, удалили строки, где возраст клиента был равен нулю, удалили строку с неопределенным полом. Выяснили, что, скорее всего, это системные ошибки, поэтому на дальнейший анализ они повлиять не могут. Дубликатов в данных было немного, мы предполагаем, что это MCAR, и это связано с процессом сбора анкет для анализа и использованием клиентами верхнего регистра при заполнении анкеты

### Лемматизация

В поле purpose явно встречаются повторяющиеся значения, которые необходимо привести к одному виду. Создаем новый дата фрейм, в который положим уникальные значения колонки purpose

In [24]:
all_purp = pd.DataFrame(data['purpose'].unique(),columns=['old_purpose'])

In [25]:
print(all_purp)

                               old_purpose
0                            покупка жилья
1                  приобретение автомобиля
2               дополнительное образование
3                          сыграть свадьбу
4                        операции с жильем
5                              образование
6                    на проведение свадьбы
7                  покупка жилья для семьи
8                     покупка недвижимости
9        покупка коммерческой недвижимости
10              покупка жилой недвижимости
11  строительство собственной недвижимости
12                            недвижимость
13              строительство недвижимости
14      на покупку подержанного автомобиля
15            на покупку своего автомобиля
16   операции с коммерческой недвижимостью
17        строительство жилой недвижимости
18                                   жилье
19         операции со своей недвижимостью
20                              автомобили
21                   заняться образованием
22        с

Лемматизируем колонку old_purpose

In [26]:
all_purp['lemmas'] = all_purp['old_purpose'].apply(m.lemmatize)

In [27]:
print(all_purp['lemmas'])

0                               [покупка,  , жилье, \n]
1                     [приобретение,  , автомобиль, \n]
2                  [дополнительный,  , образование, \n]
3                             [сыграть,  , свадьба, \n]
4                        [операция,  , с,  , жилье, \n]
5                                     [образование, \n]
6                   [на,  , проведение,  , свадьба, \n]
7             [покупка,  , жилье,  , для,  , семья, \n]
8                        [покупка,  , недвижимость, \n]
9       [покупка,  , коммерческий,  , недвижимость, \n]
10             [покупка,  , жилой,  , недвижимость, \n]
11    [строительство,  , собственный,  , недвижимост...
12                                   [недвижимость, \n]
13                 [строительство,  , недвижимость, \n]
14    [на,  , покупка,  , подержать,  , автомобиль, \n]
15         [на,  , покупка,  , свой,  , автомобиль, \n]
16    [операция,  , с,  , коммерческий,  , недвижимо...
17       [строительство,  , жилой,  , недвижимос

Посчитаем количество уникальных значений в колонке lemmas

In [28]:
b = Counter()
for a in all_purp['lemmas']:
    b += Counter(a)
print(b)


Counter({' ': 59, '\n': 38, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'подержать': 1, 'со': 1, 'подержанный': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1})


Мы видим, что самые популярные цели кредита - это недвижимость, автомобиль, образование, свадьба, ремонт


Создадим функцию, которая унифицирует цели кредита

In [29]:
def new_purpose(row):
    lemmas = row['lemmas']
    
    if 'недвижимость' in lemmas or 'жилье' in lemmas:
        return 'недвижимость'
    if 'автомобиль' in lemmas :
        return 'автомобиль'
    if 'образование' in lemmas:
        return 'образование'
    if 'свадьба' in lemmas:
        return 'свадьба'
    
    if 'ремонт' in lemmas:
        return 'ремонт'
    
    return None #если не угадаем с леммой, возвращалось None, и мы можем быстро найти ошибку


Создаем новый столбец, куда записываем новые унифицированные цели

In [30]:
all_purp['new_purpose'] = all_purp.apply(new_purpose, axis=1)

In [31]:
print(all_purp)

                               old_purpose  \
0                            покупка жилья   
1                  приобретение автомобиля   
2               дополнительное образование   
3                          сыграть свадьбу   
4                        операции с жильем   
5                              образование   
6                    на проведение свадьбы   
7                  покупка жилья для семьи   
8                     покупка недвижимости   
9        покупка коммерческой недвижимости   
10              покупка жилой недвижимости   
11  строительство собственной недвижимости   
12                            недвижимость   
13              строительство недвижимости   
14      на покупку подержанного автомобиля   
15            на покупку своего автомобиля   
16   операции с коммерческой недвижимостью   
17        строительство жилой недвижимости   
18                                   жилье   
19         операции со своей недвижимостью   
20                              ав

In [32]:
all_purp.info() # смотрим, все ли строки заполнились

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   old_purpose  38 non-null     object
 1   lemmas       38 non-null     object
 2   new_purpose  38 non-null     object
dtypes: object(3)
memory usage: 1.0+ KB


Добавляем новый столбец с новыми унифицированными целями

In [33]:
def old_to_new_purpose(row):
    a = row['purpose']
    
    b = all_purp[all_purp['old_purpose'] == a]
    return b['new_purpose'].values[0]

In [34]:
data['new_purpose'] = data.apply(old_to_new_purpose, axis=1)

In [35]:
data.info() #проверяем, что столбец добавился полностью

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21352 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21352 non-null  int64 
 1   days_employed     21352 non-null  int32 
 2   dob_years         21352 non-null  int64 
 3   education         21352 non-null  object
 4   education_id      21352 non-null  int64 
 5   family_status     21352 non-null  object
 6   family_status_id  21352 non-null  int64 
 7   gender            21352 non-null  object
 8   income_type       21352 non-null  object
 9   debt              21352 non-null  int64 
 10  total_income      21352 non-null  int32 
 11  purpose           21352 non-null  object
 12  new_purpose       21352 non-null  object
dtypes: int32(2), int64(5), object(6)
memory usage: 2.1+ MB


In [36]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,new_purpose
0,1,-8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,недвижимость
1,1,-4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,недвижимость
3,3,-4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,-926,27,высшее,0,гражданский брак,1,m,компаньон,0,255763,покупка жилья,недвижимость
6,0,-2879,43,высшее,0,женат / замужем,0,f,компаньон,0,240525,операции с жильем,недвижимость
7,0,-152,50,среднее,1,женат / замужем,0,m,сотрудник,0,135823,образование,образование
8,2,-6929,35,высшее,0,гражданский брак,1,f,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,-2188,41,среднее,1,женат / замужем,0,m,сотрудник,0,144425,покупка жилья для семьи,недвижимость


### Вывод

Таким образом, мы лемматизировали поле "цель" для последующего анализа. Мы выяснили, что  самые популярные цели кредита - это недвижимость, автомобиль, образование, свадьба, ремонт


### Категоризация данных

Теперь нам небходимо категоризировать данные для того, чтобы грамотно дать ответы на вопросы относительно возможности возврата кредита в срок пользователями. Мы выделили 4 основных параметра, которые могут повлиять: наличие детей, семейное положение, уровень дохода и цели кредита. Попробуем категоризировать данные. Начнем с наличия детей. В данных явно есть артефакты, попробуем их найти и выявить

In [37]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

Здесь мы явно видим два значения, которые не могут соответствовать истине - 1 и 20. Посмотрим, сколько заявителей указало, что у них 20 детей.

In [38]:
data[data['children'] == 20].count()

children            75
days_employed       75
dob_years           75
education           75
education_id        75
family_status       75
family_status_id    75
gender              75
income_type         75
debt                75
total_income        75
purpose             75
new_purpose         75
dtype: int64

Мы видим, что 75 человек указали, что у них 20 детей. Для выборки в 20 тысяч человек это крайне сомнительная цифра, поэтому разумным решением кажется удаление этих строчек, а также удаление строки, где количество детей = -1

In [39]:
data = data.loc[(data['children'] != 20) & (data['children'] != -1)]

In [40]:
data.info() #проверяем, удалились ли строки

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21230 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21230 non-null  int64 
 1   days_employed     21230 non-null  int32 
 2   dob_years         21230 non-null  int64 
 3   education         21230 non-null  object
 4   education_id      21230 non-null  int64 
 5   family_status     21230 non-null  object
 6   family_status_id  21230 non-null  int64 
 7   gender            21230 non-null  object
 8   income_type       21230 non-null  object
 9   debt              21230 non-null  int64 
 10  total_income      21230 non-null  int32 
 11  purpose           21230 non-null  object
 12  new_purpose       21230 non-null  object
dtypes: int32(2), int64(5), object(6)
memory usage: 2.1+ MB


Отлично, артефакты удалились. Теперь нам нужно категоризировать количество детей. Создадим новый столбец, в котором будет значение 0, если количество детей = 0, и 1, если количество детей > 0 

In [41]:
def amount_of_child(row):
    a = row['children']
    
    if a > 0:
        return 1
    if a == 0:
        return 0
    return None

Далее создаем столбец children_id в дата фрейме 

In [42]:
data['children_id'] = data.apply(amount_of_child, axis=1)

In [43]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21230 entries, 0 to 21524
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21230 non-null  int64 
 1   days_employed     21230 non-null  int32 
 2   dob_years         21230 non-null  int64 
 3   education         21230 non-null  object
 4   education_id      21230 non-null  int64 
 5   family_status     21230 non-null  object
 6   family_status_id  21230 non-null  int64 
 7   gender            21230 non-null  object
 8   income_type       21230 non-null  object
 9   debt              21230 non-null  int64 
 10  total_income      21230 non-null  int32 
 11  purpose           21230 non-null  object
 12  new_purpose       21230 non-null  object
 13  children_id       21230 non-null  int64 
dtypes: int32(2), int64(6), object(6)
memory usage: 2.3+ MB


In [44]:
data['children_id'].unique() #проверяем, что нет нанов

array([1, 0], dtype=int64)

Далее категоризируем уровень дохода, для этого используем quantile, чтобы разбить группы на следующие: до 25% - 'бедные', от 25% до 50% 'ниже среднего', от 50% до 75% 'выше среднего', от 75% и выше 'богатые'

In [45]:
data['total_income'].quantile([0.25, 0.5, 0.75])

0.25     89066.75
0.50    135758.00
0.75    195795.50
Name: total_income, dtype: float64

In [46]:
def total_income_categorized(row):
    a = row['total_income']
    
    if a <= 89066.75:
        return 'бедные'
    if a >89066.75 and a <=135758.00:
        return 'ниже среднего'
    if a > 135758.00 and a <= 195795.50:
        return 'выше среднего'
    if a >195795.50:
        return 'богатые'
    return None #проверяем, чтобы не было нанов

Создаем в дата фрейме столбец, куда будем записывать категории по доходу

In [47]:
data['total_income_catgorized'] = data.apply(total_income_categorized, axis=1)

In [48]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21230 entries, 0 to 21524
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   children                 21230 non-null  int64 
 1   days_employed            21230 non-null  int32 
 2   dob_years                21230 non-null  int64 
 3   education                21230 non-null  object
 4   education_id             21230 non-null  int64 
 5   family_status            21230 non-null  object
 6   family_status_id         21230 non-null  int64 
 7   gender                   21230 non-null  object
 8   income_type              21230 non-null  object
 9   debt                     21230 non-null  int64 
 10  total_income             21230 non-null  int32 
 11  purpose                  21230 non-null  object
 12  new_purpose              21230 non-null  object
 13  children_id              21230 non-null  int64 
 14  total_income_catgorized  21230 non-nul

In [49]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,new_purpose,children_id,total_income_catgorized
0,1,-8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,недвижимость,1,богатые
1,1,-4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,автомобиль,1,ниже среднего
2,0,-5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,недвижимость,0,выше среднего
3,3,-4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,образование,1,богатые
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,свадьба,0,выше среднего
5,0,-926,27,высшее,0,гражданский брак,1,m,компаньон,0,255763,покупка жилья,недвижимость,0,богатые
6,0,-2879,43,высшее,0,женат / замужем,0,f,компаньон,0,240525,операции с жильем,недвижимость,0,богатые
7,0,-152,50,среднее,1,женат / замужем,0,m,сотрудник,0,135823,образование,образование,0,выше среднего
8,2,-6929,35,высшее,0,гражданский брак,1,f,сотрудник,0,95856,на проведение свадьбы,свадьба,1,ниже среднего
9,0,-2188,41,среднее,1,женат / замужем,0,m,сотрудник,0,144425,покупка жилья для семьи,недвижимость,0,выше среднего


In [50]:
data['total_income_catgorized'].unique()

array(['богатые', 'ниже среднего', 'выше среднего', 'бедные'],
      dtype=object)

Далее мы хотим проверить, действительно ли уникальный family_status_id присваивается уникальному family_status, для этого далее смотрим по каждому family_status_id его family status. И мы действительно видим, что словарь верный. 

In [51]:
data[data['family_status_id'] == 0]['family_status'].unique()

array(['женат / замужем'], dtype=object)

In [52]:
data[data['family_status_id'] == 1]['family_status'].unique()

array(['гражданский брак'], dtype=object)

In [53]:
data[data['family_status_id'] == 2]['family_status'].unique()

array(['вдовец / вдова'], dtype=object)

In [54]:
data[data['family_status_id'] == 3]['family_status'].unique()

array(['в разводе'], dtype=object)

In [55]:
data[data['family_status_id'] == 4]['family_status'].unique()

array(['не женат / не замужем'], dtype=object)

### Вывод

Мы выбрали 4 основных параметра, которые могут влиять на факт возврата кредита в срок - наличие детей, цель кредита, семейное положение и уровень дохода; категоризировали данные для того, чтобы в следующем шаге проверить наши гипотезы о влиянии этого фактора на возврат долгов.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Сделаем сводную таблицу для того, чтобы понять, какая доля от всего количества заемщиков с детьми\без детей не возвращают кредит в срок. Берем в values возраст, так как нам, в принципе, без разницы, как именно посчитать количество клиентов, значение "возраст" точно заполнено, 

In [56]:
children_debt = data.pivot_table(index='children_id', columns='debt', values='dob_years', aggfunc='count')

In [57]:
children_debt.columns = ['Нет долга', 'Есть долг']

In [58]:
children_debt['Доля должников'] = (children_debt['Есть долг']/
                                (children_debt['Есть долг'] 
                                 + children_debt['Нет долга'])).map('{:.1%}'.format)

In [59]:
print(children_debt)

             Нет долга  Есть долг Доля должников
children_id                                     
0                12963       1058           7.5%
1                 6543        666           9.2%


### Вывод

Таким образом, мы обнаружили, что люди, у которых есть дети, чаще не отдают долги на 22,5% (9,2% / 7,5%)

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [60]:
family_debt = data.pivot_table(index='family_status_id', columns='debt', values='dob_years', aggfunc='count')

In [61]:
family_debt.columns = ['Нет долга', 'Есть долг']

In [62]:
family_debt['Доля должников'] = (family_debt['Есть долг']/
                                (family_debt['Есть долг'] 
                                 + family_debt['Нет долга'])).map('{:.1%}'.format)

In [63]:
print(family_debt)

                  Нет долга  Есть долг Доля должников
family_status_id                                     
0                     11290        923           7.6%
1                      3729        383           9.3%
2                       884         62           6.6%
3                      1095         84           7.1%
4                      2508        272           9.8%


### Вывод

Таким образом, мы обнаружили, что чаще всего не отдают долги люди из категории "не женат \ не замужем" и люди, состоящие в гражданском браке. В то же время, более ответственными клиентами являются люди из категории "вдова \ вдовец", возможно, это как-то связано с тем, что с потерей супруга \ супруга у них появилась бОльшая ответственность за оставшихся детей \ родственников, и из-за этого они и к долгам относятся более ответственно

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [64]:
income_debt = data.pivot_table(index='total_income_catgorized', columns='debt', values='dob_years', aggfunc='count')

In [65]:
income_debt.columns = ['Нет долга', 'Есть долг']

In [66]:
income_debt['Доля должников'] = (income_debt['Есть долг']/
                                (income_debt['Есть долг'] 
                                 + income_debt['Нет долга'])).map('{:.1%}'.format)

In [67]:
print(income_debt)

                         Нет долга  Есть долг Доля должников
total_income_catgorized                                     
бедные                        4889        419           7.9%
богатые                       4928        380           7.2%
выше среднего                 4832        475           9.0%
ниже среднего                 4857        450           8.5%


### Вывод

Таким образом, мы обнаружили, что больше других не возвращают кредиты люди из категории "доход выше среднего" (а не "бедные", как можно было бы предположить), в то же время, ответственнее остальных к своим кредитам относятся люди из категории "богатые" (разница между этими двумя категориями - 25%)

- Как разные цели кредита влияют на его возврат в срок?

In [68]:
purpose_debt = data.pivot_table(index='new_purpose', columns='debt', values='dob_years', aggfunc='count')

In [69]:
purpose_debt.columns = ['Нет долга', 'Есть долг']

In [70]:
purpose_debt['Доля должников'] = (purpose_debt['Есть долг']/
                                (purpose_debt['Есть долг'] 
                                 + purpose_debt['Нет долга'])).map('{:.1%}'.format)

In [71]:
print(purpose_debt)

              Нет долга  Есть долг Доля должников
new_purpose                                      
автомобиль         3861        397           9.3%
недвижимость       9926        777           7.3%
образование        3601        369           9.3%
свадьба            2118        181           7.9%


### Вывод

Мы выяснили, что хуже всего отдают долги клиенты, заявляющие, что берут кредит на покупку автомобиля и для оплаты образования, в то же время, цель "кредит на недвижимость", скорее всего, приведет к возврату кредита


### Шаг 4. Какие признаки больше всего влияют на то, вернет ли человек долг? Выясним с помощью Catboost

Попробуем проделать, по сути, аналогичную работу, но с помощью библиотеки Catboost

In [72]:
! pip install catboost

Поделим данные на две части - обучающая выборка и валидационная в соотношении 70 на 30
Чтобы не было warning, создадим копию и зафиксируем random state

In [73]:
train = data.sample(frac=0.7,random_state=42).copy()

In [74]:
val = data[~data.index.isin(train.index)].copy()

Поделим все признаки на фичи и целевую переменную

In [75]:
data.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose', 'new_purpose', 'children_id',
       'total_income_catgorized'],
      dtype='object')

Указываем для Catboost существующие категориальные фичи в параметре cat_features

In [76]:
X_col = ['children', 'days_employed', 'dob_years', 'education', 
       'family_status', 'gender', 'income_type',
       'total_income', 'new_purpose', 'purpose']
y_col = ['debt']
cat_features = ['education', 'family_status', 'gender', 'income_type', 'purpose', 'new_purpose']

Так как у нас задача бинарной классификации, импортируем необходимый модуль

In [77]:
from catboost import CatBoostClassifier

#verbose - чтобы выводил каждую 100-ю строку
model = CatBoostClassifier(verbose=100)

In [78]:
model.fit(train[X_col],train[y_col], #то, на чем мы тренируемся
          
          eval_set=(val[X_col],val[y_col]), #то, на чем мы валидируемся

          cat_features=cat_features #технически сообщаем, что есть категориальные переменные
          )

Learning rate set to 0.061717
0:	learn: 0.6323342	test: 0.6321707	best: 0.6321707 (0)	total: 198ms	remaining: 3m 17s
100:	learn: 0.2684484	test: 0.2697962	best: 0.2697303 (93)	total: 4.46s	remaining: 39.7s
200:	learn: 0.2588145	test: 0.2704951	best: 0.2697269 (149)	total: 10.4s	remaining: 41.5s
300:	learn: 0.2500965	test: 0.2713920	best: 0.2697269 (149)	total: 16.7s	remaining: 38.9s
400:	learn: 0.2416445	test: 0.2727065	best: 0.2697269 (149)	total: 24.1s	remaining: 36.1s
500:	learn: 0.2333495	test: 0.2733527	best: 0.2697269 (149)	total: 33.8s	remaining: 33.7s
600:	learn: 0.2248536	test: 0.2743656	best: 0.2697269 (149)	total: 51.2s	remaining: 34s
700:	learn: 0.2164397	test: 0.2757547	best: 0.2697269 (149)	total: 1m	remaining: 25.8s
800:	learn: 0.2085547	test: 0.2769611	best: 0.2697269 (149)	total: 1m 7s	remaining: 16.9s
900:	learn: 0.2020530	test: 0.2777839	best: 0.2697269 (149)	total: 1m 15s	remaining: 8.29s
999:	learn: 0.1955436	test: 0.2787579	best: 0.2697269 (149)	total: 1m 22s	rema

Прогоняем валидационную выборку

In [79]:
val['score_cat'] = model.predict_proba(val[X_col])[:,1]

Нам интересно понять, какие из фичей самые важные.

In [80]:
X_col

['children',
 'days_employed',
 'dob_years',
 'education',
 'family_status',
 'gender',
 'income_type',
 'total_income',
 'new_purpose',
 'purpose']

In [81]:
fi = pd.DataFrame({'w':model.feature_importances_,'name':X_col})

In [82]:
fi.sort_values('w',ascending=False,inplace=True)

In [83]:
fi

,w,name
1,15.735843,days_employed
2,15.048757,dob_years
3,14.536790,education
4,11.070271,family_status
5,9.128359,gender
6,8.219093,income_type
8,8.146076,new_purpose
7,7.396830,total_income
9,6.399852,purpose
0,4.318131,children


Таким образом, в нашей модели самыми важными фичами оказались количество трудовых дней, количество лет и образование

### Шаг 5. Общий вывод

Таким образом, мы провели исследование надежности заемщиков. Стоит отметить, что данные оказались достаточно "грязными" - в них присутствовали пропуски, дублирующиеся значения и артефакты. 

Касаемо причин пропуска можем выделить следующие: 
- незаполнение анкеты до конца
- некоторые вопросы были необязательными
- если сбор данных проводился через сайт или приложение, то система могла сама подставлять ответы. 

Рекомендовали бы помечать те ответы, которые являются обязательными, звездочкой или, если анкета находится на сайте или в приложении, не пускать на дальнейший этап без корректно заполненных даннных.

Дубликатов в данных было немного, мы предполагаем, что это MCAR, и это связано с процессом сбора анкет для анализа

В ходе работы над данным проектом мы обнаружили интересные инсайты. Оказалось, что самые популярные цели кредита - это недвижимость, автомобиль, образование, свадьба, ремонт.


Мы провели оценку платежеспособности клиентов, желающих получить кредит. Мы пришли к следующим выводам:
- Наиболее платежеспособной и отдающей долги аудиторией являются клиенты, у которых нет детей; вдовцы или состоящие в разводе; относящиеся по уровню дохода к категории "богатые"; заявляющие целью кредита покупку недвижимости.
- Наименее способная вернуть кредит категория клиентов - это люди с детьми; не замужем\не женаты или состоящие в гражданском браке; с уровнем дохода "выше среднего"; берущие кредит с такими целями, как "покупка автомобиля", "образование"


Также мы проанализировали, какие фичи оказываются самыми важными при предсказании долга у потребителя. Этими фичами оказались стаж работы, возраст человека и его образование.